Tensor is the basic lego block in tensorflow. It is somewhat similar to arrays in numpy or dataframe in pandas. The rank of a tensor is its number of dimension. 

Ex: <li>5 is a tensor with Rank 0 and shape [] <br/>
    <li>[1, 2] is a tensor with rank 1 and shape [2] <br/>
    <li>[[1, 2, 3], [3, 4, 5]] is a tensor with rank 2 and shape [2, 3] <br/>
    <li>[[[1, 2, 3]], [[4, 5, 6]]] is a tensor with rank 3 and shape [2, 1, 3] <br/>

One of the building block in tensorflow is <b>computational graph</b>. It can be thought of as a map of operations to be performed for the anlaysis<br/>
So, the computational graphs can be thought of as black-box models which take some tensors as inputs and produces the relevant tensor outputs. <br/>

Since, each black-box model consists of many operations. We say that computational graph is a graph of nodes. Each node takes no/some tensors as input and produces an output. Hence, we can assume nodes to be equivalent to functions in python. Note: This node can be a constant as well which outputs a given value let's say 1.

The process of implementing TensorFlow Core can be broken down into two parts: 
<li>Building the computational graph<br/>
<li>Running the computational graph

An example of building nodes in the computational graph is below:

In [ ]:
import tensorflow as tf
import numpy as np
node1 = tf.constant(3.0)
node2 = tf.constant([2, 3], tf.float32)
print(node1, node2)

Note: As we told earlier, the process of building and running the computational graph are kept as two separate process in tensorflow. Hence, when we are printing the nodes we can't print their value. Now, let's see how we can run the computational graphs

To run the computational graphs, we have to create a Session object and use its run method to evaluate the node values as follows: 

In [ ]:
s = tf.Session()
print(s.run([node1, node2]))

Now, lets try something bit more involed. 

In [ ]:
node3 = node1 + node2
s.run(node3)

<i>Note: that the broadcasting rules of numpy are valid here</i> 

Since, we compared the nodes to functions, they should have the capabililty to take any input. To bring in this functionality, we have placeholder. Placeholder can be thought of as empty tensor whose value will be provided at the run of computational graph. 

In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
node3 = a + b
print(a, b, node3)

Now when we run the session, we will provide the values of placeholders by specifying their values using a dictionary. This dictionary is known as feed_dict in the Tensor world ;)

In [ ]:
fd = {a: 3, b:[2, 3]}
print(s.run(node3, fd))

The third type of tensors available in tensorflow are Variables. The value of variables can changes during the run of the computational graphs. They are generally used to define the parameters of the model which we want to train. Ex. 

In [ ]:
W = tf.Variable([2.0], tf.float32)
b = tf.Variable([0.0], tf.float32)
x = tf.placeholder(tf.float32)
print(W, b, x)

There are two major differences between constant and variable tensors.
<li> The value of constant tensors can not change during the run of computational graph whereas the value of variable tensors change during the run of computational graph
<li> The value of constant tensor is initialised when we define them whereas to initialise the variable tensors we have to call a special function 'tf.global_variables_initializer()'

In [ ]:
init = tf.global_variables_initializer()
s.run(init)
print(s.run(b))

Now, lets give it a try and see it evaluates values. 

In [ ]:
linear_model = W * x + b
d = {x: [[1, 2], [3, 4]]}
output = s.run(linear_model, d)
print(output)

<i>Note: The type conversion is a big pain in the ass in tensorflow. So, try to explicitly mention the type of every tensor. Also, note how we can pass array of any dimension as x in the feed_dict 

There is still one thing missing from the overall basic architecture of a machine learning setup. We are not learning/training the weights currently. For that, we will require a 'y' placeholder and a loss function to improve the 'W' variable and 'b' tensors iteratively.  

In [ ]:
y = tf.placeholder(tf.float32)
loss = tf.reduce_sum((linear_model - y)**2, axis = None)
print(loss)
print(s.run(loss, {x: [1, 2, 3], y: [3, 4, 4]}))

Now, we want to minimize this loss by using some kind of an optimizer algorithm like gradient descent or stochastic gradient descent. This process is achieved by using the train API in tensorflow. <p>If we are not using the tensorflow, we would have to calculate the gradients of the loss and then re-evaluate the value of W and b based on our algorithm. This process is very cumbersome, entirely mechanical and error-prone. The beauty of tensorflow is that you can implement your algorithm without worrying about the optimizing part. They have a variety of in-built functions to provide that functionality. For example, 

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01 is the learning rate
e = optimizer.minimize(loss)
print(e)

The way to read the above code is:<br>
Take gradient descent as our optimizer with learning rate of 0.01. After this we specify the loss function 'loss' and the objective i.e to minimize the loss in the second line of code.Note, optimizer is a way to get gradient w.r.t to the variable tensors involed with the loss function. Hence, we need to specify the loss function and the objective in the second line. 
<p> After defining the optimizer and objective function, we just need to iterate through the input values long enough to train the values for W and b. This is what we achieve below: 

In [ ]:
s.run(init)
for i in range(1000):
    _, weights, b_value, l = s.run([e, W, b, loss], {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    # This piece is code is just to track the progress of the model  
    if i % 100 == 0:
        print(weights, b_value, l)

print(s.run([W, b]))

Once, we have trained the model, we would like to predict the y values based on x values. For this, we need to pick the best value of W and b and assign it to W and b. This can be achieved by using tf.assign function.  

In [ ]:
s.run(init)
asg_W_best = tf.assign(W, weights)
asg_b_best = tf.assign(b, b_value)
s.run([asg_W_best, asg_b_best])
print(s.run(loss, {x: [1, 2], y: [0, -1]}))

Note: Even after initializing the value of W and b at the top, we reassign the values of W and b in line 2 & 3.  

The entire code can be found below:

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

#==============================================================================
# Building the computational graph
#
#==============================================================================

# Initialising the tensors
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
W = tf.Variable([-1.0], tf.float32)
b = tf.Variable([0.0], tf.float32)

# Initialising the model and its loss function
linear_model = x * W + b
loss = tf.reduce_sum((linear_model - y)**2)

# Defining the optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
training = optimizer.minimize(loss)

#==============================================================================
# Running the computational graph
#
#==============================================================================

init = tf.global_variables_initializer()
s = tf.Session()
s.run(init)
data = {x: [1,2, 3, 4], y: [3, 4, 5, 6]}
for i in range(1000):
    s.run(training, data)

best_W, best_b, best_loss = s.run([W, b, loss], data)
print(best_W, best_b, best_loss)